## BlackJack

In [1]:
#定義基本參數
import random
from IPython.display import clear_output
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen',
         'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10,
         'Jack':10,'Queen':10, 'King':10, 'Ace':11}

In [2]:
#定義卡片
class Card:

    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
    
    def __str__(self):
        return self.rank + ' of ' + self.suit

#定義新牌組    
class Deck:

    def __init__(self):
        self.deck = [] #從empty list開始
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))

    def __str__(self):
        deck_comp = ""
        for card in self.deck:
            deck_comp += '\n' +card.__str__()
        return "The deck has:" +deck_comp

    def shuffle(self):
        random.shuffle(self.deck)
    
    def deal(self):
        single_card = self.deck.pop()
        return single_card

#定義手牌    
class Hand:
    def __init__(self):
        self.cards = [] #從empty list開始
        self.value = 0 #從zero value開始
        self.aces = 0 #稍後追蹤ace

    #加入手牌及計算value
    def add_card(self,card):
        self.cards.append(card)
        self.value += values[card.rank]
        if card.rank == 'Ace':
            self.aces += 1 #ace數量計算

    #追蹤ace
    def adjust_for_ace(self):
        while self.value >21 and self.aces: #(Python預設=0為False,>0為True)
            self.value -= 10
            self.aces -= 1

#定義輸贏籌碼
class Chips:

    def __init__(self,balance): 
        self.total = balance
        self.bet = 0
    
    def win_bet(self):
        self.total += self.bet

    def lose_bet(self):
        self.total -= self.bet

    def is_game_over(self):
        return self.total <= 0

In [3]:
#遊戲中輸入籌碼
def take_bet(chips):
    
    while True:
        try:
            chips.bet = int(input("How many chips would you like to bet? : "))
        except ValueError:
            print("Sry, bet must be an interger!")
        else:
            if chips.bet > chips.total:
                print("Sry,your bet can't exceed",chips.total)
            else:
                break
                
#遊戲中選擇抽牌後行為
def hit(deck,hand):

    hand.add_card(deck.deal()) #從新牌組抽一張加入手中牌組
    hand.adjust_for_ace() #追蹤是不是多了ace

#遊戲中選擇是否抽牌
def hit_or_stand(deck,hand):
    global playing #控制接下來while loop
    
    while True:
        x = input("Would you like to Hit or Stand? (Enter 'h' or 's') : ")

        if x[0].lower() == 'h':
            hit(deck,hand)
        elif x[0].lower() == 's':
            print("Player stands.Dealer turns.")
            playing = False
        else:
            print("Sry,plz enter 'h' or 's'.")
            continue
        break

#遊戲中顯示莊家玩家卡牌面板
def show_some(player,dealer):
    print("\nDealer's hand:\n <card hidden>\n ",dealer.cards[1])
    print("\nPlayer's hand:", *player.cards, sep="\n ")

#遊戲結束顯示莊家玩家卡牌面板
def show_all(player,dealer):
    print("\nDealer's hand:", *dealer.cards, sep="\n ")
    print("\nPlayer's hand:", *player.cards, sep="\n ")



In [4]:
#輸贏平手行為
def player_busts(player,dealer,chips):
    print("Player busts!")
    chips.lose_bet()

def player_wins(player,dealer,chips):
    print("Player wins!")
    chips.win_bet()

def dealer_busts(player,dealer,chips):
    print("Dealer busts!")
    chips.win_bet()

def dealer_wins(player,dealer,chips):
    print("Dealer wins!")
    chips.lose_bet()

def push(player,dealer):
    print("It's a tie! push")

## 遊戲邏輯設定

In [6]:
#籌碼初始設定
balance = 100
player_chips = Chips(balance)
#進入遊戲
playing = True

while True:
    print("Welcome to BlackJack!\n\
    Get as close to 21 as you can without going over!\n\
    Dealer hits until reaching 17.\nAces count as 1 or 11.")

    #導入新牌組,shuffle
    deck = Deck()
    deck.shuffle()

    #兩方各發兩張牌
    player_hand = Hand() #empty list
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())

    dealer_hand = Hand() #empty list
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())

    #玩家下注
    take_bet(player_chips)

    #遊戲中顯示手牌
    show_some(player_hand,dealer_hand)

    #遊戲開始行為
    while playing: 

        #玩家選擇是否繼續發牌至手上
        hit_or_stand(deck,player_hand) #hit則繼續,stand則使playing = False終止while loop

        #繼續發牌的話顯示卡片
        clear_output()
        show_some(player_hand,dealer_hand)

        #發牌後如玩家爆掉,扣除籌碼
        if player_hand.value > 21:
            player_busts(player_hand,dealer_hand,player_chips)
            break

    #玩家選停止發牌,跳出上方while loop,莊家抽牌
    if player_hand.value <= 21:

        while dealer_hand.value <17:
            hit(deck,dealer_hand)

        #揭漏所有手牌
        show_all(player_hand,dealer_hand)

        if dealer_hand.value > 21: #莊家爆掉
            dealer_busts(player_hand,dealer_hand,player_chips)
        elif dealer_hand.value > player_hand.value: #莊家比玩家接近21
            dealer_wins(player_hand,dealer_hand,player_chips)
        elif dealer_hand.value < player_hand.value: #玩家比莊家接近21
            player_wins(player_hand,dealer_hand,player_chips)
        else:
            push(player_hand,dealer_hand) #平局

    #玩家籌碼
    print(f"Player now have {player_chips.total} chips.")
    
    #檢查籌碼數量是否能繼續玩
    if player_chips.is_game_over():
        print("Game over! You have no more chips.")
        break

    #再玩一局
    while True:
        new_game = input("Would you like to play another hand? (Enter 'y' or 'n') : ")
        if new_game[0].lower() == 'y':
            playing = True
            break
        elif new_game[0].lower() == 'n':
            print("Thanks for playing,bye!")
            playing = False
            break
        else:
            print("Sry,plz enter 'y' or 'n'.")
            continue
    if playing == False:
        break


Dealer's hand:
 <card hidden>
  Six of Diamonds

Player's hand:
 Nine of Diamonds
 King of Diamonds

Dealer's hand:
 King of Hearts
 Six of Diamonds
 Jack of Hearts

Player's hand:
 Nine of Diamonds
 King of Diamonds
Dealer busts!
Player now have 100 chips.
Would you like to play another hand? (Enter 'y' or 'n') : n
Thanks for playing,bye!
